# Step 1: Scraping

In [2]:
# Dependencies
from bs4 import BeautifulSoup
import splinter 
from splinter import Browser
import requests
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import pandas as pd

In [3]:
def news_titles_description ():
    url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
    html = requests.get(url)

    # Create BeautifulSoup object; parse with 'html.parser'
    soup = BeautifulSoup(html.text, 'html.parser')
    results = soup.find()

    # Collect the latest News Title
    titles = results.find_all('div', class_="content_title")

    for title in titles:
        news_title = title.a.text.strip()

    # Collect the paragraph texts
    papagraphs = results.find_all('div', class_="rollover_description_inner")

    for p in papagraphs:
        news_p = p.text.strip()


# JPL Mars Space Images - Featured Image

- Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.


- Make sure to find the image url to the full size .jpg image.


- Make sure to save a complete url string for this image.

Source: https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars

In [41]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\navyh\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [42]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [43]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve all articles tags
# articles = soup.find_all('div', class_="carousel_items")

results = soup.find()
# print(results.prettify())

In [44]:
browser.find_by_id("full_image").click()

In [45]:
browser.find_by_text("more info     ").click()

In [46]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
featured_img = soup.find("img", class_='main_image')['src']
featured_img

'/spaceimages/images/largesize/PIA19330_hires.jpg'

In [47]:
# image_url =re.search(r'/(.*?)\'', article).group(1)

In [49]:
base_url = 'https://www.jpl.nasa.gov'
featured_image_url = base_url + featured_img

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19330_hires.jpg


# Mars Facts

- Scrape the table containing facts about the planet including Diameter, Mass, etc.

- Use Pandas to convert the data to a HTML table string.
- Source: https://space-facts.com/mars/

In [10]:
url = 'https://space-facts.com/mars/'
html = requests.get(url)

In [11]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html.text, 'html.parser')

In [12]:
results = soup.find()
print(results.prettify())

<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://space-facts.com/wp/xmlrpc.php" rel="pingback"/>
  <link as="font" crossorigin="" href="https://space-facts.com/wp-content/themes/gridbox/assets/fonts/roboto-slab-v7-latin-ext_latin-700.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://space-facts.com/wp-content/themes/gridbox/assets/fonts/roboto-v18-latin-ext_latin-regular.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://space-facts.com/wp-content/themes/gridbox/assets/fonts/roboto-v18-latin-ext_latin-700.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://space-facts.com/wp-content/themes/gridbox/assets/fonts/roboto-v18-latin-ext_latin-italic.woff2" rel="preload" type="font/woff2"/>
  <meta content="OI26qBRJ2SI673vDUn_KgAd_H

In [13]:
table = results.find('table', class_="tablepress tablepress-id-p-mars", id="tablepress-p-mars")
trs = table.find_all("tr")
tds_column1 = table.find_all('td', class_='column-1')
tds_column2 = table.find_all('td', class_='column-2')

# Get values in column 1
column1 = []

for td in tds_column1:
    if (td):
    
        try:
            column1.append(td.text)

        except:
            pass

# Get values in column 2
column2 = []

for td in tds_column2:
    if (td):
    
        try:
            column2.append(td.text)

        except:
            pass
        
print(column1)
print('---------------------------------------------------------------------')
print(column2)



['Equatorial Diameter:', 'Polar Diameter:', 'Mass:', 'Moons:', 'Orbit Distance:', 'Orbit Period:', 'Surface Temperature: ', 'First Record:', 'Recorded By:']
---------------------------------------------------------------------
['6,792 km', '6,752 km', '6.39 × 10^23 kg (0.11 Earths)', '2 (Phobos & Deimos)', '227,943,824 km (1.38 AU)', '687 days (1.9 years)', '-87 to -5 °C', '2nd millennium BC', 'Egyptian astronomers']


In [15]:
data = list(zip(column1,column2))

# Import column1 and column2 into Pandas DataFrame
mars_fact_df = pd.DataFrame(data, columns=['Parameter', 'Value']).set_index('Parameter')
mars_fact_df

,Value
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [17]:
# Convert table to html format before displaying in html
mars_fact_html = mars_fact_df.to_html(header=True, index=False)

# Mars Hemispheres

- Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.


- You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.


- Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.


- Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.


- Source: https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars

In [47]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
html = requests.get(url)

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html.text, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="//ajax.googleapis.com/ajax/libs/jqueryui/1.11.3/themes/smoothness/jquery-ui.css" rel="stylesheet" type="text/css"/>
  <title>
   Astropedia Search Results | USGS Astrogeology Science Center
  </title>
  <meta content="USGS Astrogeology Science Center Astropedia search results." name="description"/>
  <meta content="USGS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="x61hXXVj7wtfBSNOPnTftajMsZ5yB2W-qRoyr7GtOKM" name="google-site-verification"/>
  <!--<link rel="stylesheet" href="http://fonts.googleapis.com/css?family=Open+Sans:400italic,400,bold"/>-->
  <link href="/css/main.css" media="screen" rel="stylesheet"/>
  <link href="/css/print.css" media="pr

In [52]:
results = soup.find_all('a', class_='itemLink product-item')
results

[<a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/><div class="description"><h3>Cerberus Hemisphere Enhanced</h3></div></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/><div class="description"><h3>Schiaparelli Hemisphere Enhanced</h3></div></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/syrtis_major_enhanced"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/><div class="description"><h3>Syrtis Major Hemisphere Enhanced</h3></div></a>,
 <a class="itemLink product-item" href="/search

In [88]:
# Extract all 4 links/href from the results

# titles = []
img_urls = []
hemisphere_image_urls = []
merge_dict = {}
title_dict = {}
img_url_dict = {}

for result in results:
    title = result.text
    tail = result.get('href')
    full_url = f'https://astrogeology.usgs.gov{tail}'
    
    # Open each link found using get request
    mar_html = requests.get(full_url)
    mars_soup = BeautifulSoup(mar_html.text, 'html.parser')
    
    # Once each link is opened, searcj for the full resolution image url link
    img_url = mars_soup.find("div", class_="downloads").a.get('href')
    
    titles.append(title)
    img_urls.append(img_url)
    
    keys_1 = ['title', 'title', 'title', 'title']
    keys_2 = ['img_url', 'img_url', 'img_url', 'img_url']

    
    # Create 1st dictionary
    for key in keys_1:
        for value in titles:
            title_dict[key] = value
    
    # Create  2nd dictionary

    for key in keys_2:
        for value in img_urls:
            img_url_dict[key] = value
    
    # Merge 2 dictionaries
    def Merge(img_url_dict,title_dict):
        res = { **img_url_dict, **title_dict}
        return res
    
    merge_dict = Merge(title_dict,img_url_dict)
    
    # Append the merge dictionaries into a list
    hemisphere_image_urls.append(merge_dict)
    
pprint(hemisphere_image_urls)

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]


In [ ]:
# converting your Jupyter notebook into a Python script called scrape_mars.py with a function
# called scrape that will execute all of your scrapingcode from above 
# and return one Python dictionary containing all of the scraped data.

def scrape():
    